In [ ]:
# @title
# =======================================================
# 🩺 Healthcare Symptom Checker – using Groq API
# =======================================================

!pip install --quiet requests gradio

import requests
import json
import gradio as gr

# — Ask user for Groq API key —
groq_api_key = input("Enter your Groq API key: ").strip()
GROQ_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"

# Function to query Groq API
def check_symptoms_with_groq(symptoms, age=None, sex=None, comorbidities=None):
    if not symptoms.strip():
        return "⚠️ Please enter your symptoms."

    # System + user prompt
    system_prompt = """You are a healthcare assistant that provides
educational, non-diagnostic suggestions of possible conditions and next steps based on symptoms.
Always:
- Use cautious, probabilistic language ("possible", "could be").
- Flag red-flag symptoms and advise urgent care where needed.
- Do not provide prescriptions.
Return JSON with keys:
  suggestions (list of {condition, confidence_percent, rationale}),
  triage (string),
  red_flags (list),
  disclaimer (string)
"""

    user_prompt = f"""Symptoms: {symptoms}
Age: {age or 'unknown'}
Sex: {sex or 'unknown'}
Comorbidities: {comorbidities or 'none specified'}"""

    headers = {
        "Authorization": f"Bearer {groq_api_key}",
        "Content-Type": "application/json"
    }

    body = {
        "model": "llama-3.3-70b-versatile",  # ✅ Supported model
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.2,
        "max_tokens": 600
    }

    # Call the API
    resp = requests.post(GROQ_ENDPOINT, headers=headers, json=body)
    if resp.status_code != 200:
        return f"❌ API error {resp.status_code}: {resp.text}"

    data = resp.json()
    content = data.get("choices", [{}])[0].get("message", {}).get("content", "")

    # Try parse JSON
    try:
        start = content.index("{")
        end = content.rindex("}") + 1
        parsed = json.loads(content[start:end])
    except Exception:
        return f"⚠️ Could not parse JSON. Raw output:\n{content}"

    # Format output for display
    out = "## 🩺 Healthcare Symptom Checker Results\n"
    out += "**Disclaimer:** " + parsed.get("disclaimer", "Educational only; not a diagnosis.") + "\n\n"
    out += "### 🔎 Possible Conditions\n"
    for s in parsed.get("suggestions", []):
        out += f"- **{s.get('condition')}** ({s.get('confidence_percent')}%) — {s.get('rationale')}\n"
    out += "\n### 🧭 Triage Advice\n" + parsed.get("triage", "N/A") + "\n"
    if parsed.get("red_flags"):
        out += "\n### 🚨 Red Flags\n" + ", ".join(parsed["red_flags"]) + "\n"
    return out


# Gradio interface
demo = gr.Interface(
    fn=check_symptoms_with_groq,
    inputs=[
        gr.Textbox(label="Symptoms", placeholder="e.g. fever, cough, fatigue"),
        gr.Textbox(label="Age (optional)"),
        gr.Textbox(label="Sex (optional)"),
        gr.Textbox(label="Comorbidities (optional)")
    ],
    outputs=gr.Markdown(),
    title="Symptom Checker using Groq",
    description="Enter symptoms to get educational possible conditions & next steps. ⚠️ Not medical advice."
)

# Launch app
demo.launch(share=True, inbrowser=True, debug=True)


Enter your Groq API key: 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
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5fb87e79fad92b9fc4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
